In [36]:
import numpy as np
np.set_printoptions(precision=6, suppress=True)

In [153]:
x_0 = np.array([
    [2], 
    [3], 
    [1], 
    [2], 
    [5], 
])

M = np.array([
    [1,1], 
    [1,0], 
    [0,1], 
    [0,1], 
    [0,1], 
])

M_ = np.array([
    [1 , 0, 0, 0], 
    [-1, 0, 0, 0], 
    [-1, 0, 1, 1], 
    [0 , 1,-1,-1], 
    [0 ,-1, 0, 0], 
])


S = np.array([
    [1,0 ], 
    [1,0 ], 
    [1 ,0 ], 
    [0 ,1 ], 
    [0 ,1 ], 
])


D = np.array([
    [0 ,1 ], 
    [1 ,0 ],  
])


m_c = np.array([
    [0.2,], 
    [0.3,],
])

q_c = np.array([
    [-3,], 
    [1,],
])

a = -np.log(np.array([
    [4], 
    [1], 
    [2], 
    [1], 
    [3], 
]))

k = np.array([
    [1], 
    [0.7], 
    [0.1], 
    [0],  
])

v = np.array([
    [0], 
    [0], 
    [0], 
    [0.8],  
])

c = np.array([
    [0.1], 
    [0.1], 
    [0.1], 
    [0.2], 
    [0.1], 
])


x_h = np.array([
    [0], 
    [0], 
    [0], 
    [0], 
    [3], 
])


h = np.array([
    [0], 
    [0], 
    [0], 
    [0], 
    [2], 
])

###

p_0 = np.array([
    [0], 
    [0], 
    [0], 
    [0], 
    [0], 
])

In [154]:
x, p = x_0, p_0
q =  np.dot(  np.diag( np.dot(M, q_c).T[0] ), S )
V = np.dot( q, np.dot(1/(D+1)-1, q.T) )
m = np.dot( M, m_c)

In [173]:
def relu(x) :
    return np.maximum(0,x)

def Energy(x,p) :
    global a, V, m
    
    energy = np.dot(x.T, np.log(x)) + np.dot(a.T, x) + 0.5*np.dot(x.T, np.dot(V, x)) + 0.5*(np.dot( (1/m.T), (p*p*x)))
    
    return energy[0]

def Flow(x,p) :
    global M_, k, v, h, x_h, c
    
    grad_x = (np.log(x)+1).T + a.T + np.dot(x.T, V) + (0.5*(1/m)*p*p).T
    grad_p = (0.5*(1/m)*p*x).T
    
    chemical_flow_x = -np.dot(M_, (k.T*(np.exp( np.dot(grad_x, M_)) -1 )*(np.exp(-0.5*np.dot(grad_x, M_) + 0.5*np.dot(np.log(x.T), np.abs(M_)))) ).T)
    hamiltonian_flow_x = np.dot(M_, v*(np.dot(grad_p, M_).T) )
    hamiltonian_flow_p = -np.dot(M_, v*(np.dot(grad_x, M_).T) )
    colision_flow_p = -c*(grad_p).T
    external_homeostasis_flow_x = h*(x_h-x)
    
    return chemical_flow_x + hamiltonian_flow_x,  hamiltonian_flow_p + colision_flow_p
    #return chemical_flow_x + hamiltonian_flow_x + external_homeostasis_flow_x,  hamiltonian_flow_p + colision_flow_p

In [174]:
flow_x, flow_p = Flow(x,p) 
flow_x, flow_p

(array([[  6.137168],
        [ -6.137168],
        [ 26.554393],
        [-33.095706],
        [  0.404145]]),
 array([[-0.      ],
        [-0.      ],
        [ 8.709035],
        [-8.709035],
        [-0.      ]]))

In [170]:
energy = Energy(x,p)

In [171]:
np.dot(M.T, flow_x)

array([[ 0.],
       [-4.]])

In [95]:
np.diag( np.dot(M, q_c).T[0] )

array([[-2,  0,  0,  0,  0],
       [ 0, -3,  0,  0,  0],
       [ 0,  0,  1,  0,  0],
       [ 0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1]])

In [94]:
np.dot(M, q_c).T[0]

array([-2, -3,  1,  1,  1])

In [78]:
Q = np.dot(M,m_q)*S

In [79]:
M_ = -M_
V = np.dot(np.dot(Q,1/(D+1)-1) , Q.T)

m = np.dot(M,m_c)

G_x = np.log(x.T)+a.T+np.dot(x.T, V) + (p*p/(2*m)).T

dx_dt = -np.dot(M_,(k.T*(np.exp(np.dot(G_x,M_))-1)*np.exp(-0.5*np.dot(G_x,M_))*np.exp(0.5*np.dot(np.log(x.T),np.absolute(M_)))).T)

dx_dt

array([[   2.828427],
       [  -2.828427],
       [ 226.012498],
       [-228.898661],
       [   0.057735]])

In [62]:
np.dot(x.T, V)

array([[ 7. , 10.5, -3.5,  6. ,  6. ]])

In [22]:
np.dot(np.dot(x.T, V), M_)

array([[ 0. , -9.5,  0. , -9.5]])

In [81]:
M_ = -M_
(np.exp(np.dot(G_x,M_))-1)*np.exp(-0.5*np.dot(G_x,M_))*np.exp(0.5*np.dot(np.log(x.T),np.absolute(M_)))

array([[  2.828427, 326.915608,  -0.57735 , 326.915608]])

In [47]:
M_ = -M_
((np.exp(np.dot(G_x,M_))-1)*np.exp(np.dot(G_x,relu(-M_)))).T

array([[1096.633158],
       [ 806.842488],
       [-134.476264],
       [ 806.842488]])

In [53]:
np.dot(np.dot(x.T, V), relu(-M_))

array([[ 7. , -3.5,  6. , -3.5]])

In [81]:
dx_dt

array([[ 1.        ],
       [-1.        ],
       [ 0.05      ],
       [-1.08333333],
       [ 0.03333333]])

In [66]:
(np.exp(np.dot(G_x,M_))-1)

array([[-0.87737352, -0.75      ,  0.2       , -0.75      ]])

In [67]:
np.dot(S.T,relu(M_))

array([[-2,  1,  0,  1],
       [ 0,  0,  1,  0]])

In [68]:
np.dot(S.T,relu(M_))

array([[-2,  1,  0,  1],
       [ 0,  0,  1,  0]])

In [69]:
V = np.dot(np.dot(S,1/(D+1)-1) , S.T)

In [41]:
V

array([[ 0. ,  0. ,  0. ,  1. ,  1. ],
       [ 0. ,  0. ,  0. ,  1.5,  1.5],
       [ 0. ,  0. ,  0. , -0.5, -0.5],
       [ 1. ,  1.5, -0.5,  0. ,  0. ],
       [ 1. ,  1.5, -0.5,  0. ,  0. ]])

In [27]:
temp = np.dot(x.T,V)

In [30]:
temp

array([[ 7. , 10.5, -3.5,  6. ,  6. ]])

In [39]:
np.dot(temp, relu(M_))

array([[ 7. , -3.5,  6. , -3.5]])

In [38]:
np.dot(temp, relu(-M_))

array([[7., 6., 6., 6.]])

In [9]:
a = np.array([1,2])
b = np.array([3,4])
np.dot(a,b,a)

ValueError: output array is not acceptable (must have the right datatype, number of dimensions, and be a C-Array)